In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [ ]:
#checking fot device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cpu


In [ ]:
#Transforms
transformer=transforms.Compose(
    [
        transforms.Resize((150,150)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), #changes the color from 0-255 to 0-1
        transforms.Normalize([0.5,0.5,0.5], #normalizee from 0-1 to -1-1, formula to create new pixules (x-mean)/std
                             [0.5,0.5,0.5])
    ]
)

In [ ]:
#Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#Path
train_path = '/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN'
test_path = '/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TEST'

In [ ]:
#DataLoader
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=50, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=50, shuffle=True
)

In [ ]:
#catagories
root=pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['downdog', 'goddess', 'plank', 'tree', 'warrior2']


In [ ]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=5):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
        #((w-f+2P)/s) +1

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (50,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (50,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (50,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (50,12,75,75)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (50,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (50,20,75,75)



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (50,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (50,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (50,32,75,75)


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)



        #Feed forwad function

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)


            #Above output will be in matrix form, with shape (256,32,75,75)

        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output

In [ ]:
model = ConvNet(num_classes=5).to(device)

In [ ]:
#Optimizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count,test_count)

999 420


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:

#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        #outputs=model(images)
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count


    # Evaluation on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 0 Train Loss: tensor(23.6499) Train Accuracy: 0.3123123123123123 Test Accuracy: 0.6642857142857143
Epoch: 1 Train Loss: tensor(5.2355) Train Accuracy: 0.6576576576576577 Test Accuracy: 0.6452380952380953
Epoch: 2 Train Loss: tensor(2.3573) Train Accuracy: 0.8138138138138138 Test Accuracy: 0.7857142857142857
Epoch: 3 Train Loss: tensor(0.9198) Train Accuracy: 0.9309309309309309 Test Accuracy: 0.7738095238095238
Epoch: 4 Train Loss: tensor(0.6199) Train Accuracy: 0.9619619619619619 Test Accuracy: 0.7571428571428571
Epoch: 5 Train Loss: tensor(1.0111) Train Accuracy: 0.9369369369369369 Test Accuracy: 0.8166666666666667
Epoch: 6 Train Loss: tensor(0.3286) Train Accuracy: 1.014014014014014 Test Accuracy: 0.8571428571428571
Epoch: 7 Train Loss: tensor(0.2549) Train Accuracy: 1.039039039039039 Test Accuracy: 0.8428571428571429
Epoch: 8 Train Loss: tensor(0.2014) Train Accuracy: 1.038038038038038 Test Accuracy: 0.7928571428571428
Epoch: 9 Train Loss: tensor(0.0682) Train Accuracy: 1.066

-----------

In [ ]:
!pip install -q jmd_imagescraper

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.4 MB/s eta 0:00:00


In [ ]:
from jmd_imagescraper.core import * # dont't worry, it's designed to work with import *

duckduckgo_search(root, "yoga_dataset", "yoga warrior2", max_results=10)
duckduckgo_search(root, "yoga_dataset", "yoga goddess", max_results=10)
duckduckgo_search(root, "yoga_dataset", "yoga plank", max_results=10)
duckduckgo_search(root, "yoga_dataset", "yoga tree", max_results=10)
duckduckgo_search(root, "yoga_dataset", "yoga downdog", max_results=10)

Duckduckgo search: yoga warrior2


Duckduckgo search: yoga goddess


Duckduckgo search: yoga plank


Duckduckgo search: yoga tree


Duckduckgo search: yoga downdog


[PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/041_03b2a077.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/042_13d84224.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/043_12dc7875.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/044_9f5b5932.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/045_f536bfec.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/046_91762918.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/047_e499caf4.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/048_22e3018a.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET/TRAIN/yoga_dataset/049_83d9122d.jpg'),
 PosixPath('/content/gdrive/MyDrive/YOGA/content/cleaned/DATASET

In [ ]:
ZIP_NAME = "imges" # maybe change this?

!rm -f {ZIP_NAME}

In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=5)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=5, bias=True)
)

In [ ]:
#prediction function
def prediction(img_path,transformer):

    image=Image.open(img_path)

    image_tensor=transformer(image).float()


    image_tensor=image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    input=Variable(image_tensor)


    output=model(input)

    index=output.data.numpy().argmax()

    pred=classes[index]

    return pred


In [ ]:
prediction('/content/gdrive/MyDrive/00000010.jpg',transformer)

'tree'